## Import modules

In [1]:
import sys
sys.path.append('src')

In [2]:
import pandas
from itertools import product
from energiapy.components.resource import Resource
from energiapy.components.scenario import Scenario 
from energiapy.components.horizon import Horizon
from energiapy.components.process import Process
from energiapy.model.specialparams.dataset import DataSet
from energiapy.model.specialparams.unbound import BigM
from energiapy.utils.data_utils import make_henry_price_df, remove_outliers

In [3]:
def load_data(loc:str, index:list):
    df = pandas.read_csv(f'data/{loc}_solar19.csv')
    df['idx'] = index
    df = df.set_index('idx')
    return df
idx = [(i,j,k) for i,j,k in product(range(1), range(365), range(24))]
weather_ny = load_data('ny', idx)[['DNI', 'Wind Speed']]
weather_ny = weather_ny.rename(columns= {'DNI': 'dni', 'Wind Speed': 'wind_speed'})
weather_sd = load_data('sd', idx)[['dni', 'wind_speed']]
weather_ho = load_data('ho', idx)[['dni', 'wind_speed']]
demand_sd = pandas.read_excel('data/HistoricalEMSHourlyLoad-2019.xlsx', index_col= 0)[['SDGE']]
demand_ho = pandas.read_excel('data/Native_Load_2019.xlsx')[['COAST']]
demand_ny = pandas.DataFrame(pandas.read_csv('data/NYC_load.csv')['Load']) #from Will and Doga
ng_price = make_henry_price_df(
    file_name='data/Henry_Hub_Natural_Gas_Spot_Price_Daily.csv', year=2020)
ng_price = ng_price.drop(columns= 'scales')
ng_price = remove_outliers(ng_price, sd_cuttoff = 3)
ng_price = pandas.concat([ng_price])

In [4]:
s = Scenario()

In [5]:
s.h = Horizon(discretizations=[365, 24])

In [6]:
a = pandas.DataFrame({'a': list(range(8760))})
b = pandas.DataFrame({'b': list(range(8760))})
cc = DataSet(data = a, horizon= s.h, scaling_max= True)

In [7]:
s.H2 = Resource(discharge = {s.t0: [0,4], s.t1: [22, 34], s.t2: (a, b)}, sell_cost = (0, 10))


In [8]:
# Resource.__hash__ 

In [9]:
# s.H2.discharge.cons()

In [10]:
s.Solar = Resource(gwp =50, eutt = 60, consume=BigM, basis='MW', label='Solar Power')
s.Wind = Resource(consume=cc, basis='MW', label='Wind Power')
s.Power = Resource(basis='MW', label='Power generated')
s.Uranium = Resource(consume=pandas.DataFrame({'a': [i for i in range(365)]}), purchase_cost= 42.70/(250/2), basis='kg', label='Uranium')
# s.H2 = Resource(basis='tons', label='Hydrogen')
s.H2_L = Resource(discharge = (0,23), basis='tons', label='Hydrogen')
s.CO2_AQoff = Resource(basis='tons', label='Carbon dioxide - sequestered')
s.H2O = Resource(consume=(20,50), purchase_cost= cc, basis='tons', label='Water')
s.CH4 = Resource(consume=[20, 40], purchase_cost=20, basis='tons', label='Natural gas')
s.CO2 = Resource(basis='tons', label='Carbon dioxide', block='Resource')
s.CO2_Vent = Resource(basis='tons', label='Carbon dioxide - Vented', sell_cost= (2, 20))
s.O2 = Resource(discharge= [20, BigM], basis='tons', label='Oxygen')
s.CO2_DAC = Resource(basis='tons', label='Carbon dioxide - captured')


In [11]:
s.resources 

[CO2,
 CH4,
 H2,
 CO2_DAC,
 Wind,
 Solar,
 Uranium,
 H2_L,
 CO2_AQoff,
 O2,
 CO2_Vent,
 H2O,
 Power]

## Declare processes

In [ ]:
# s.LiI = Process(store= 2000, conversion = {s.Power: 0.9}, store_loss = 0.2, store_cost= 0.4, capex = 3516428, fopex= 87910, vopex = 0, capacity= [1000,BigM], label='Lithium-ion battery', basis = 'MW')
s.WF = Process(gwp = 50, odp = 2900,  produce= weather_ho[['wind_speed']],  conversion={s.Power: {s.Wind: -1}},capex=1462000, fopex=43000, vopex=4953, capacity=1000, land_use = 30, label='Wind mill array', basis = 'MW')
s.PV = Process(sell_cost= {s.Power:30}, credit = {s.Power: pandas.DataFrame({'a': [2]})}, conversion={s.Power: {1:{s.Solar: -1.2}, 2: {s.Solar: -1.5}}}, capex = 1333262, fopex= 22623, capacity=True, label = 'Solar PV', basis = 'MW')
s.SMRH = Process(land_use=50, conversion={s.H2: {s.Power: -1.11, s.CH4: -3.76, s.H2O: -23.7, s.CO2_Vent: 1.03, s.CO2: 9.332}}, capex =2520000, fopex = 945000, vopex = 51.5, capacity= 1000, label='Steam methane reforming + CCUS')
s.NGCC = Process(purchase_cost= {s.CH4: 4, s.H2O: 20}, conversion = {s.Power: {s.CH4: -0.108, s.H2O: -10, s.CO2_Vent: 0.297*0.05, s.CO2:0.297*0.95}}, capex = 2158928, fopex= 53320, vopex = 4090, capacity= [1,100], label = 'NGCC + 95% CC')
s.SMR = Process(capex = 2400, fopex = 800, vopex = 0.03,  conversion={s.H2: {s.Power: -1.11, s.CH4: -3.76, s.H2O: -23.7, s.CO2_Vent: 9.4979}}, capacity=1000, label='Steam methane reforming')
s.H2FC = Process(purchase_cost= {s.H2: 2}, discharge= {s.Power: 20}, conversion = {s.Power: {s.H2:-0.050}}, capex =  1.6*10**6, vopex = 3.5, capacity = 1000, label = 'hydrogen fuel cell')
s.DAC = Process(capex = 0.02536, fopex = 0.634, conversion= {s.CO2_DAC: {s.Power: -0.193, s.H2O: -4.048}}, capacity=1000, label='Direct air capture')
# s.PSH = Process(conversion = {s.Power: 0.6}, capex = 3924781, fopex= 17820, vopex = 512.5, store = 10000, capacity=1000, label='Pumped storage hydropower', basis = 'MW')
s.ASMR = Process(consume = {s.Uranium: 40}, conversion={s.Power:{s.Uranium: -4.17*10**(-5), s.H2O: -3.364}}, capex = 7988951, fopex= 0.04*0.730, capacity = 1000, label='Small modular reactors (SMRs)')
s.AWE = Process(incidental= 20, land_use = (0,20), conversion= {s.H2: {s.Power: -1, s.O2: 0.7632, s.H2O: -0.1753}}, capex = 1.1*10**6, fopex = 16918, capacity = BigM, label='Alkaline water electrolysis (AWE)', citation='Demirhan et al. 2018 AIChE paper') 


AttributeError: 'Scenario' object has no attribute 'H2'

In [ ]:
s.WF.consume.cons()

consume('Wind', 'WF', 't0')<=M


In [ ]:
s.WF.produce.cons()

produce('Power', 'WF', 't2')<=capacity('Power', 'WF', 't2')


In [ ]:
s.Power.produce.cons()

produce('Power', 'ASMR', 't0')<=capacity('Power', 'ASMR', 't0')
produce('Power', 'H2FC', 't0')<=capacity('Power', 'H2FC', 't0')
produce('Power', 'NGCC', 't0')<=capacity('Power', 'NGCC', 't0')
produce('Power', 'PV', 't0')<=capacity('Power', 'PV', 't0')
produce('Power', 'WF', 't2')<=capacity('Power', 'WF', 't2')


In [ ]:
s.Power.cons()

consume('Power', 'AWE', 't0')<=M
consume('Power', 'DAC', 't0')<=M
consume('Power', 'SMR', 't0')<=M
consume('Power', 'SMRH', 't0')<=M
credit('Power', 'PV', 't0')=DSet|Credit('Power', 'PV', 't0')|*discharge('Power', 'PV', 't0')
discharge('Power', 'ASMR', 't0')<=M
discharge('Power', 'H2FC', 't0')=20
discharge('Power', 'NGCC', 't0')<=M
discharge('Power', 'PV', 't0')<=M
discharge('Power', 'WF', 't0')<=M
produce('Power', 'ASMR', 't0')<=capacity('Power', 'ASMR', 't0')
produce('Power', 'H2FC', 't0')<=capacity('Power', 'H2FC', 't0')
produce('Power', 'NGCC', 't0')<=capacity('Power', 'NGCC', 't0')
produce('Power', 'PV', 't0')<=capacity('Power', 'PV', 't0')
produce('Power', 'WF', 't2')<=capacity('Power', 'WF', 't2')
sell_cost('Power', 'PV', 't0')=30*discharge('Power', 'PV', 't0')


In [ ]:
s.cons()

capacity('ASMR', 't0')=1000
capacity('AWE', 't0')<=M
capacity('DAC', 't0')=1000
capacity('H2FC', 't0')=1000
capacity('NGCC', 't0')<=100
capacity('NGCC', 't0')>=1
capacity('PV', 't0')<=M
capacity('SMR', 't0')=1000
capacity('SMRH', 't0')=1000
capacity('WF', 't0')=1000
capex('ASMR', 't0')=7988951*capacity('ASMR', 't0')
capex('AWE', 't0')=1100000.0*capacity('AWE', 't0')
capex('DAC', 't0')=0.02536*capacity('DAC', 't0')
capex('H2FC', 't0')=1600000.0*capacity('H2FC', 't0')
capex('NGCC', 't0')=2158928*capacity('NGCC', 't0')
capex('PV', 't0')=1333262*capacity('PV', 't0')
capex('SMR', 't0')=2400*capacity('SMR', 't0')
capex('SMRH', 't0')=2520000*capacity('SMRH', 't0')
capex('WF', 't0')=1462000*capacity('WF', 't0')
consume('CH4', 'NGCC', 't0')<=M
consume('CH4', 'SMR', 't0')<=M
consume('CH4', 'SMRH', 't0')<=M
consume('CH4', 't0')<=40
consume('CH4', 't0')>=20
consume('H2', 'H2FC', 't0')<=M
consume('H2O', 'ASMR', 't0')<=M
consume('H2O', 'AWE', 't0')<=M
consume('H2O', 'DAC', 't0')<=M
consume('H2O', 'N

In [ ]:
s.WF.produce.cons()

produce('Power', 'WF', 't2')<=capacity('Power', 'WF', 't2')


In [ ]:
s.ASMR.produce.cons()

produce('Power', 'ASMR', 't0')<=capacity('Power', 'ASMR', 't0')


In [ ]:
s.ASMR.cons()

capacity('ASMR', 't0')=1000
capex('ASMR', 't0')=7988951*capacity('ASMR', 't0')
consume('H2O', 'ASMR', 't0')<=M
consume('Uranium', 'ASMR', 't0')=40
discharge('Power', 'ASMR', 't0')<=M
fopex('ASMR', 't0')=0.0292*capacity('ASMR', 't0')
produce('Power', 'ASMR', 't0')<=capacity('Power', 'ASMR', 't0')


In [ ]:
for  i in s.processes:
    i.produce.cons()

produce('H2', 'SMR', 't0')<=capacity('H2', 'SMR', 't0')
produce('Power', 'ASMR', 't0')<=capacity('Power', 'ASMR', 't0')
produce('Power', 'NGCC', 't0')<=capacity('Power', 'NGCC', 't0')
produce('Power', 'H2FC', 't0')<=capacity('Power', 'H2FC', 't0')
produce('Power', 'PV', 't0')<=capacity('Power', 'PV', 't0')
produce('H2', 'AWE', 't0')<=capacity('H2', 'AWE', 't0')
produce('H2', 'SMRH', 't0')<=capacity('H2', 'SMRH', 't0')
produce('Power', 'WF', 't2')<=capacity('Power', 'WF', 't2')
produce('CO2_DAC', 'DAC', 't0')<=capacity('CO2_DAC', 'DAC', 't0')


In [ ]:
s.Power.cons()

consume('Power', 'AWE', 't0')<=M
consume('Power', 'DAC', 't0')<=M
consume('Power', 'SMR', 't0')<=M
consume('Power', 'SMRH', 't0')<=M
credit('Power', 'PV', 't0')=DSet|Credit('Power', 'PV', 't0')|*discharge('Power', 'PV', 't0')
discharge('Power', 'ASMR', 't0')<=M
discharge('Power', 'H2FC', 't0')=20
discharge('Power', 'NGCC', 't0')<=M
discharge('Power', 'PV', 't0')<=M
discharge('Power', 'WF', 't0')<=M
produce('Power', 'ASMR', 't0')<=capacity('Power', 'ASMR', 't0')
produce('Power', 'H2FC', 't0')<=capacity('Power', 'H2FC', 't0')
produce('Power', 'NGCC', 't0')<=capacity('Power', 'NGCC', 't0')
produce('Power', 'PV', 't0')<=capacity('Power', 'PV', 't0')
produce('Power', 'WF', 't2')<=capacity('Power', 'WF', 't2')
sell_cost('Power', 'PV', 't0')=30*discharge('Power', 'PV', 't0')


In [ ]:
s.WF.produce.cons()

produce('Power', 'WF', 't2')<=capacity('Power', 'WF', 't2')


In [ ]:
s.ASMR.produce.cons()

produce('Power', 'ASMR', 't0')<=capacity('Power', 'ASMR', 't0')


In [ ]:
s.cons

In [24]:
s.WF.produce.cons()

produce('Power', 'WF', 't2')<=capacity('Power', 'WF', 't2')


In [25]:
for i in s.resources:
    if hasattr(i, 'consume') and getattr(i, 'consume') is not None:
        i.consume.cons()

consume('Uranium', 'ASMR', 't0')=40
consume('Uranium', 't1')=DSet|Consume('Uranium', 't1')|
consume('Wind', 'WF', 't0')<=M
consume('Wind', 't2')=DSet|Consume('Wind', 't2')|
consume('Power', 'AWE', 't0')<=M
consume('Power', 'DAC', 't0')<=M
consume('Power', 'SMR', 't0')<=M
consume('Power', 'SMRH', 't0')<=M
consume('CH4', 'NGCC', 't0')<=M
consume('CH4', 'SMR', 't0')<=M
consume('CH4', 'SMRH', 't0')<=M
consume('CH4', 't0')<=40
consume('CH4', 't0')>=20
consume('H2O', 'ASMR', 't0')<=M
consume('H2O', 'AWE', 't0')<=M
consume('H2O', 'DAC', 't0')<=M
consume('H2O', 'NGCC', 't0')<=M
consume('H2O', 'SMR', 't0')<=M
consume('H2O', 'SMRH', 't0')<=M
consume('H2O', 't0')=Th|Consume('H2O', 't0')|, Th in [20, 50]
consume('Solar', 'PV', 't0')<=M
consume('Solar', 't0')<=M
consume('H2', 'H2FC', 't0')<=M


In [26]:
s.processes

[NGCC, SMRH, SMR, WF, ASMR, PV, H2FC, DAC, AWE]

In [27]:
s.Power.produce.cons()

produce('Power', 'ASMR', 't0')<=capacity('Power', 'ASMR', 't0')
produce('Power', 'H2FC', 't0')<=capacity('Power', 'H2FC', 't0')
produce('Power', 'NGCC', 't0')<=capacity('Power', 'NGCC', 't0')
produce('Power', 'PV', 't0')<=capacity('Power', 'PV', 't0')
produce('Power', 'WF', 't2')<=capacity('Power', 'WF', 't2')


In [28]:
s.ASMR.produce.cons()

produce('Power', 'ASMR', 't0')<=capacity('Power', 'ASMR', 't0')


In [29]:
for i in s.processes:
    print(i)
    i.produce.cons()
    print('___________')

NGCC
produce('Power', 'NGCC', 't0')<=capacity('Power', 'NGCC', 't0')
___________
SMRH
produce('H2', 'SMRH', 't0')<=capacity('H2', 'SMRH', 't0')
___________
SMR
produce('H2', 'SMR', 't0')<=capacity('H2', 'SMR', 't0')
___________
WF
produce('Power', 'WF', 't2')<=capacity('Power', 'WF', 't2')
___________
ASMR
produce('Power', 'ASMR', 't0')<=capacity('Power', 'ASMR', 't0')
___________
PV
produce('Power', 'PV', 't0')<=capacity('Power', 'PV', 't0')
___________
H2FC
produce('Power', 'H2FC', 't0')<=capacity('Power', 'H2FC', 't0')
___________
DAC
produce('CO2_DAC', 'DAC', 't0')<=capacity('CO2_DAC', 'DAC', 't0')
___________
AWE
produce('H2', 'AWE', 't0')<=capacity('H2', 'AWE', 't0')
___________


In [ ]:
s.WF.produce.cons()

produce('Power', 'WF', 't2')<=capacity('Power', 'WF', 't2')


In [18]:
s.WF.cons()

produce('Power', 'WF', 't2')<=capacity('Power', 'WF', 't2')
produce('Power', 'WF', 't2')<=capacity('Power', 'WF', 't2')
discharge('Power', 'WF', 't0')<=M
consume('Wind', 't2')=DSet|Consume('Wind', 't2')|
consume('Wind', 'WF', 't0')<=M
capacity('WF', 't0')=1000
land_use('WF', 't0')=30*capacity('WF', 't0')
capex('WF', 't0')=1462000*capacity('WF', 't0')
fopex('WF', 't0')=43000*capacity('WF', 't0')
vopex('WF', 't0')=4953*produce('WF', 't0')
gwp('WF', 't0')=50*capacity('WF', 't0')
gwp('WF', 't0')=50*consume('WF', 't0')
gwp('WF', 't0')=50*discharge('WF', 't0')
odp('WF', 't0')=2900*capacity('WF', 't0')
odp('WF', 't0')=2900*consume('WF', 't0')
odp('WF', 't0')=2900*discharge('WF', 't0')


In [17]:
type(s.WF.produce)

energiapy.model.aspect.AspectDict

In [15]:
s.WF.produce.component

WF

In [17]:
s.Power.cons()

produce('Power', 'WF', 't2')<=capacity('Power', 'WF', 't2')
discharge('Power', 'WF', 't0')<=M
produce('Power', 'PV', 't0')<=capacity('Power', 'PV', 't0')
discharge('Power', 'PV', 't0')<=M
sell_cost('Power', 'PV', 't0')=30*discharge('Power', 'PV', 't0')
credit('Power', 'PV', 't0')=DSet|Credit('Power', 'PV', 't0')|*discharge('Power', 'PV', 't0')
consume('Power', 'SMRH', 't0')<=M
produce('Power', 'NGCC', 't0')<=capacity('Power', 'NGCC', 't0')
discharge('Power', 'NGCC', 't0')<=M
consume('Power', 'SMR', 't0')<=M
produce('Power', 'H2FC', 't0')<=capacity('Power', 'H2FC', 't0')
discharge('Power', 'H2FC', 't0')=20
consume('Power', 'DAC', 't0')<=M
produce('Power', 'ASMR', 't0')<=capacity('Power', 'ASMR', 't0')
discharge('Power', 'ASMR', 't0')<=M
consume('Power', 'AWE', 't0')<=M


In [16]:
s.cons()

capacity('ASMR', 't0')=1000
capacity('AWE', 't0')<=M
capacity('DAC', 't0')=1000
capacity('H2FC', 't0')=1000
capacity('NGCC', 't0')<=100
capacity('NGCC', 't0')>=1
capacity('PV', 't0')<=M
capacity('SMR', 't0')=1000
capacity('SMRH', 't0')=1000
capacity('WF', 't0')=1000
capex('ASMR', 't0')=7988951*capacity('ASMR', 't0')
capex('AWE', 't0')=1100000.0*capacity('AWE', 't0')
capex('DAC', 't0')=0.02536*capacity('DAC', 't0')
capex('H2FC', 't0')=1600000.0*capacity('H2FC', 't0')
capex('NGCC', 't0')=2158928*capacity('NGCC', 't0')
capex('PV', 't0')=1333262*capacity('PV', 't0')
capex('SMR', 't0')=2400*capacity('SMR', 't0')
capex('SMRH', 't0')=2520000*capacity('SMRH', 't0')
capex('WF', 't0')=1462000*capacity('WF', 't0')
consume('CH4', 't0')<=40
consume('CH4', 't0')>=20
consume('H2O', 't0')=Th|Consume('H2O', 't0')|, Th in [20, 50]
consume('Solar', 't0')<=M
consume('Uranium', 't1')=DSet|Consume('Uranium', 't1')|
consume('Wind', 't2')=DSet|Consume('Wind', 't2')|
discharge('CO2_Vent', 't0')<=M
discharge('H

In [ ]:
s.H2.discharge.cons()

discharge('H2', 'AWE', 't0')<=M


In [ ]:
s.SMR.discharge

{H2: True, CO2_Vent: True}

In [ ]:
s.cons()

capacity('ASMR', 't0')=1000
capacity('AWE', 't0')<=M
capacity('DAC', 't0')=1000
capacity('H2FC', 't0')=1000
capacity('NGCC', 't0')<=100
capacity('NGCC', 't0')>=1
capacity('PV', 't0')<=M
capacity('SMR', 't0')=1000
capacity('SMRH', 't0')=1000
capacity('WF', 't0')=1000
capex('ASMR', 't0')=7988951*capacity('ASMR', 't0')
capex('AWE', 't0')=1100000.0*capacity('AWE', 't0')
capex('DAC', 't0')=0.02536*capacity('DAC', 't0')
capex('H2FC', 't0')=1600000.0*capacity('H2FC', 't0')
capex('NGCC', 't0')=2158928*capacity('NGCC', 't0')
capex('PV', 't0')=1333262*capacity('PV', 't0')
capex('SMR', 't0')=2400*capacity('SMR', 't0')
capex('SMRH', 't0')=2520000*capacity('SMRH', 't0')
capex('WF', 't0')=1462000*capacity('WF', 't0')
consume('CH4', 't0')<=40
consume('CH4', 't0')>=20
consume('H2O', 't0')=Th|Consume('H2O', 't0')|, Th in [20, 50]
consume('Solar', 't0')<=M
consume('Uranium', 't1')=DSet|Consume('Uranium', 't1')|
consume('Wind', 't2')=DSet|Consume('Wind', 't2')|
discharge('CO2_Vent', 't0')<=M
discharge('H

In [1]:
s.AWE.produce 

NameError: name 's' is not defined

In [ ]:
s.H2.parameters

[Discharge_lb('H2', 't0'),
 Discharge_lb('H2', 't1'),
 Discharge_ub('H2', 't0'),
 Discharge_ub('H2', 't1'),
 Th|Discharge('H2', 't2')|,
 Th|Sell_cost('H2', 't0')|,
 Produce('H2', 'SMRH', 't0'),
 Discharge('H2', 'SMRH', 't0'),
 Produce('H2', 'SMRH', 't0'),
 Discharge('H2', 'SMRH', 't0'),
 Produce('H2', 'SMR', 't0'),
 Discharge('H2', 'SMR', 't0'),
 Produce('H2', 'SMR', 't0'),
 Discharge('H2', 'SMR', 't0'),
 Consume('H2', 'H2FC', 't0'),
 Consume('H2', 'H2FC', 't0'),
 Purchase_cost('H2', 'H2FC', 't0'),
 Produce('H2', 'AWE', 't0'),
 Discharge('H2', 'AWE', 't0'),
 Produce('H2', 'AWE', 't0'),
 Discharge('H2', 'AWE', 't0')]

In [ ]:
s.H2.cons()

discharge('H2', 't0')<=4
discharge('H2', 't0')>=0
discharge('H2', 't1')<=34
discharge('H2', 't1')>=22
discharge('H2', 't2')=Th|Discharge('H2', 't2')|, Th in [DSet|Discharge_lb('H2', 't2')|, DSet|Discharge_ub('H2', 't2')|]
sell_cost('H2', 't0')=Th|Sell_cost('H2', 't0')|*discharge('H2', 't0'), Th in [0, 10]
produce('H2', 'SMRH', 't0')<=capacity('H2', 'SMRH', 't0')
discharge('H2', 'SMRH', 't0')<=M
produce('H2', 'SMRH', 't0')<=capacity('H2', 'SMRH', 't0')
discharge('H2', 'SMRH', 't0')<=M
produce('H2', 'SMR', 't0')<=capacity('H2', 'SMR', 't0')
discharge('H2', 'SMR', 't0')<=M
produce('H2', 'SMR', 't0')<=capacity('H2', 'SMR', 't0')
discharge('H2', 'SMR', 't0')<=M
consume('H2', 'H2FC', 't0')<=M
consume('H2', 'H2FC', 't0')<=M
purchase_cost('H2', 'H2FC', 't0')=2*consume('H2', 'H2FC', 't0')
produce('H2', 'AWE', 't0')<=capacity('H2', 'AWE', 't0')
discharge('H2', 'AWE', 't0')<=M
produce('H2', 'AWE', 't0')<=capacity('H2', 'AWE', 't0')
discharge('H2', 'AWE', 't0')<=M


In [ ]:
s.AWE.cons()

capacity('AWE', 't0')<=M
land_use('AWE', 't0')=Th|Land_use('AWE', 't0')|*capacity('AWE', 't0'), Th in [0, 20]
capex('AWE', 't0')=1100000.0*capacity('AWE', 't0')
fopex('AWE', 't0')=16918*capacity('AWE', 't0')
incidental('AWE', 't0')=20


In [ ]:
s.AWE.consume

{Power: True, H2O: True}

In [ ]:
s.H2.produce.cons()

produce('H2', 'AWE', 't0')<=capacity('H2', 'AWE', 't0')


In [ ]:
s.AWE.produce 

{H2: 1}

In [ ]:
s.AWE.cons()

capacity('AWE', 't0')<=M
land_use('AWE', 't0')=Th|Land_use('AWE', 't0')|*capacity('AWE', 't0'), Th in [0, 20]
capex('AWE', 't0')=1100000.0*capacity('AWE', 't0')
fopex('AWE', 't0')=16918*capacity('AWE', 't0')
incidental('AWE', 't0')=20


In [ ]:
s.H2.produce.cons()

produce('H2', 'AWE', 't0')<=capacity('H2', 'AWE', 't0')


In [23]:
s.cons()

capacity('ASMR', 't0')=1000
capacity('AWE', 't0')<=M
capacity('DAC', 't0')=1000
capacity('H2FC', 't0')=1000
capacity('NGCC', 't0')<=100
capacity('NGCC', 't0')>=1
capacity('PV', 't0')<=M
capacity('SMR', 't0')=1000
capacity('SMRH', 't0')=1000
capacity('WF', 't0')=1000
capex('ASMR', 't0')=7988951*capacity('ASMR', 't0')
capex('AWE', 't0')=1100000.0*capacity('AWE', 't0')
capex('DAC', 't0')=0.02536*capacity('DAC', 't0')
capex('H2FC', 't0')=1600000.0*capacity('H2FC', 't0')
capex('NGCC', 't0')=2158928*capacity('NGCC', 't0')
capex('PV', 't0')=1333262*capacity('PV', 't0')
capex('SMR', 't0')=2400*capacity('SMR', 't0')
capex('SMRH', 't0')=2520000*capacity('SMRH', 't0')
capex('WF', 't0')=1462000*capacity('WF', 't0')
consume('CH4', 't0')<=40
consume('CH4', 't0')>=20
consume('H2O', 't0')=Th|Consume('H2O', 't0')|, Th in [20, 50]
consume('Solar', 't0')<=M
consume('Uranium', 't1')=DSet|Consume('Uranium', 't1')|
consume('Wind', 't2')=DSet|Consume('Wind', 't2')|
discharge('CO2_Vent', 't0')<=M
discharge('H

In [17]:
Process.aspects()

[<Limit.CAPACITY: 3>,
 <Land.LAND_USE: 1>,
 <CashFlow.CAPEX: 6>,
 <CashFlow.FOPEX: 7>,
 <CashFlow.VOPEX: 8>,
 <CashFlow.INCIDENTAL: 9>,
 <CapBound.PRODUCE: 1>,
 <CapBound.STORE: 2>,
 <Emission.GWP: 1>,
 <Emission.ODP: 2>,
 <Emission.ACID: 3>,
 <Emission.EUTT: 4>,
 <Emission.EUTF: 5>,
 <Emission.EUTM: 6>,
 <Life.INTRODUCE: 1>,
 <Life.RETIRE: 2>,
 <Life.LIFETIME: 3>,
 <Life.PFAIL: 4>,
 <Life.TRL: 5>,
 <Limit.DISCHARGE: 1>,
 <Limit.CONSUME: 2>,
 <CashFlow.SELL_COST: 1>,
 <CashFlow.PURCHASE_COST: 2>,
 <CashFlow.STORE_COST: 3>,
 <CashFlow.CREDIT: 4>,
 <CashFlow.PENALTY: 5>]

In [18]:
s.Power.consume.cons()

consume('Power', 'AWE', 't0')<=M


In [ ]:
s.Power.discharge.cons()  

discharge('Power', 'ASMR', 't0')<=M


In [13]:
s.Power.discharge.__dict__ 

{'aspect': <Limit.DISCHARGE: 1>,
 'component': Power,
 'name': 'discharge(Power)',
 'parameters': [Discharge('Power', 'ASMR', 't0')],
 'variables': [discharge('Power', 'ASMR', 't0')],
 'constraints': [discharge('Power', 'ASMR', 't0')<=M]}

In [24]:
s.SMR.discharge 

{H2: True, CO2_Vent: True}

In [20]:
s.WF.produce.__dict__ 

{'aspect': <CapBound.PRODUCE: 1>,
 'component': WF,
 'name': 'produce(WF)',
 'parameters': [DSet|Produce('WF', 't2')|],
 'variables': [produce('WF', 't2')],
 'constraints': [produce('WF', 't2')<=capacity('WF', 't2')]}

In [12]:
Process.aspects()

[<Limit.CAPACITY: 3>,
 <Land.LAND_USE: 1>,
 <CashFlow.CAPEX: 6>,
 <CashFlow.FOPEX: 7>,
 <CashFlow.VOPEX: 8>,
 <CashFlow.INCIDENTAL: 9>,
 <CapBound.PRODUCE: 1>,
 <CapBound.STORE: 2>,
 <Emission.GWP: 1>,
 <Emission.ODP: 2>,
 <Emission.ACID: 3>,
 <Emission.EUTT: 4>,
 <Emission.EUTF: 5>,
 <Emission.EUTM: 6>,
 <Life.INTRODUCE: 1>,
 <Life.RETIRE: 2>,
 <Life.LIFETIME: 3>,
 <Life.PFAIL: 4>,
 <Life.TRL: 5>]

In [15]:
Process.resource_aspects()

[<Limit.DISCHARGE: 1>,
 <Limit.CONSUME: 2>,
 <CashFlow.SELL_COST: 1>,
 <CashFlow.PURCHASE_COST: 2>,
 <CashFlow.STORE_COST: 3>,
 <CashFlow.CREDIT: 4>,
 <CashFlow.PENALTY: 5>]

In [12]:
s.WF.discharge

{Power: True}

In [13]:
s.WF.produce

produce(WF)

{Wind: True}

In [28]:
s.WF.consume 

{Wind: True}

In [20]:
s.Wind.consume.__dict__ 

{'aspect': <Limit.CONSUME: 2>,
 'component': Wind,
 'name': 'consume(Wind)',
 'parameters': [Consume('Wind', 'WF', 't0')],
 'variables': [consume('Wind', 'WF', 't0')],
 'constraints': [consume('Wind', 'WF', 't0')<=M]}

In [21]:
s.WF.consume 

{Wind: True}

In [15]:
s.WF.produce.__dict__ 

{'aspect': <CapBound.PRODUCE: 1>,
 'component': WF,
 'name': 'produce(WF)',
 'parameters': [DSet|Produce('WF', 't2')|],
 'variables': [produce('WF', 't2')],
 'constraints': [produce('WF', 't2')<=capacity('WF', 't2')]}

In [12]:
Process.__dataclass_fields__

{'conversion': Field(name='conversion',type='IsConv',default=<dataclasses._MISSING_TYPE object at 0x000002625AACA010>,default_factory=<dataclasses._MISSING_TYPE object at 0x000002625AACA010>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),kw_only=False,_field_type=_FIELD),
 'capacity': Field(name='capacity',type='IsLimit',default=<dataclasses._MISSING_TYPE object at 0x000002625AACA010>,default_factory=<dataclasses._MISSING_TYPE object at 0x000002625AACA010>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),kw_only=False,_field_type=_FIELD),
 'land_use': Field(name='land_use',type='IsLand',default=None,default_factory=<dataclasses._MISSING_TYPE object at 0x000002625AACA010>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),kw_only=False,_field_type=_FIELD),
 'material_cons': Field(name='material_cons',type='IsMatCons',default=None,default_factory=<dataclasses._MISSING_TYPE object at 0x000002625AACA010>,init=True,repr=True,hash=N

In [35]:
s.ASMR.__dict__

{'conversion': Conv(Power,None),
 'capacity': capacity(ASMR),
 'land_use': None,
 'material_cons': None,
 'store': None,
 'produce': {Power: 1},
 'capex': capex(ASMR),
 'fopex': fopex(ASMR),
 'vopex': None,
 'incidental': None,
 'capex_pwl': None,
 'gwp': None,
 'odp': None,
 'acid': None,
 'eutt': None,
 'eutf': None,
 'eutm': None,
 'introduce': None,
 'retire': None,
 'lifetime': None,
 'pfail': None,
 'trl': None,
 'discharge': {Power: True},
 'consume': {Uranium: 40, H2O: True},
 'store_loss': None,
 'sell_cost': None,
 'purchase_cost': None,
 'store_cost': None,
 'credit': None,
 'penalty': None,
 'basis': None,
 'block': None,
 'citation': None,
 'label': 'Small modular reactors (SMRs)',
 'varying': None,
 'prod_max': None,
 'prod_min': None,
 'named': True,
 'name': 'ASMR',
 'horizon': h,
 'parameters': [Capacity('ASMR', 't0'),
  Capex('ASMR', 't0'),
  Fopex('ASMR', 't0')],
 'variables': [capacity('ASMR', 't0'),
  capex('ASMR', 't0'),
  fopex('ASMR', 't0')],
 'constraints': [ca

In [36]:
s.WF.conversion.__dict__ 

{'conversion': {Power: {Wind: -1}},
 'process': WF,
 'produce': Power,
 'modes': None,
 'n_modes': 1,
 'discharge': [Power],
 'consume': [Wind],
 'balance': {Power: 1, Wind: -1},
 'involve': [Power, Wind],
 'name': 'Conv(Power,None)'}

In [14]:
s.PV.conversion.__dict__ 

{'conversion': {Power: {1: {Solar: -1.2}, 2: {Solar: -1.5}}},
 'process': PV,
 'name': 'Conv(Power,None)',
 'produce': Power,
 'n_modes': 2,
 'modes': [1, 2],
 'discharge': [Power],
 'consume': [Solar],
 'involve': [Power, Solar],
 'balance': {1: {Power: 1, Solar: -1.2}, 2: {Power: 1, Solar: -1.5}}}

In [69]:
s.LiI.store_cost

{Power: 0.4}

In [15]:
s.cons()


discharge('H2', 't0')<=4
discharge('H2', 't0')>=0
discharge('H2', 't1')<=34
discharge('H2', 't1')>=22
discharge('H2', 't2')=Th|Discharge('H2', 't2')|, Th in [DSet|Discharge_lb('H2', 't2')|, DSet|Discharge_ub('H2', 't2')|]
sell_cost('H2', 't0')=Th|Sell_cost('H2', 't0')|*discharge('H2', 't0'), Th in [0, 10]
consume('Solar', 't0')<=M
gwp('Solar', 't0')=50*capacity('Solar', 't0')
gwp('Solar', 't0')=50*consume('Solar', 't0')
gwp('Solar', 't0')=50*discharge('Solar', 't0')
eutt('Solar', 't0')=60*capacity('Solar', 't0')
eutt('Solar', 't0')=60*consume('Solar', 't0')
eutt('Solar', 't0')=60*discharge('Solar', 't0')
consume('Wind', 't2')=DSet|Consume('Wind', 't2')|
consume('Uranium', 't1')=DSet|Consume('Uranium', 't1')|
purchase_cost('Uranium', 't0')=0.3416*consume('Uranium', 't0')
discharge('H2_L', 't0')=Th|Discharge('H2_L', 't0')|, Th in [0, 23]
consume('H2O', 't0')=Th|Consume('H2O', 't0')|, Th in [20, 50]
purchase_cost('H2O', 't2')=DSet|Purchase_cost('H2O', 't2')|*consume('H2O', 't2')
consume('

In [19]:
s.LiI_d.store_loss 

{Power_in_LiI: {Power: 0.2}}

In [18]:
s.LiI_d.store_cost

{Power_in_LiI: {Power: 0.4}}

In [14]:
s.LiI.conversion.conversion

{Power_in_LiI: {Power: 0.9}}

In [15]:
s.LiI_d.conversion.conversion

{Power: {Power_in_LiI: -1}}

In [16]:
s.processes

[LiI_d, LiI, WF, PV, SMRH, NGCC, SMR, H2FC, DAC, PSH_d, PSH, ASMR, AWE]

In [14]:
s.Power_in_LiI.cons()

discharge('Power_in_LiI', 'LiI', 't0')<=M
produce('Power_in_LiI', 'LiI', 't0')<=capacity('', 'LiI', 't0')
consume('Power_in_LiI', 'LiI_d', 't0')<=M


In [20]:
s.Power.cons()

consume('Power', 'LiI', 't0')<=M
store_cost('Power', 'LiI', 't0')=0.4*store('Power', 'LiI', 't0')
store_loss('Power', 'LiI', 't0')=0.2*store('Power', 'LiI', 't0')
discharge('Power', 'LiI_d', 't0')<=M
produce('Power', 'LiI_d', 't0')<=capacity('', 'LiI_d', 't0')
discharge('Power', 'WF', 't0')<=M
produce('Power', 'WF', 't0')<=capacity('', 'WF', 't0')
sell_cost('Power', 'PV', 't0')=30*discharge('Power', 'PV', 't0')
credit('Power', 'PV', 't0')=DSet|Credit('Power', 'PV', 't0')|*discharge('Power', 'PV', 't0')
produce('Power', 'PV', 't0')<=capacity('', 'PV', 't0')
consume('Power', 'SMRH', 't0')<=M
discharge('Power', 'NGCC', 't0')<=M
produce('Power', 'NGCC', 't0')<=capacity('', 'NGCC', 't0')
consume('Power', 'SMR', 't0')<=M
discharge('Power', 'H2FC', 't0')=20
produce('Power', 'H2FC', 't0')<=capacity('', 'H2FC', 't0')
consume('Power', 'DAC', 't0')<=M
consume('Power', 'PSH', 't0')<=M
discharge('Power', 'PSH_d', 't0')<=M
produce('Power', 'PSH_d', 't0')<=capacity('', 'PSH_d', 't0')
discharge('Power

In [16]:
s.LiI_d.__dict__ 

{'conversion': Conv(Power,None),
 'capacity': Capacity(LiI_d),
 'produce': {Power: 1},
 'land_use': None,
 'material_cons': None,
 'capex': None,
 'pwl': None,
 'fopex': None,
 'vopex': None,
 'incidental': None,
 'gwp': None,
 'odp': None,
 'acid': None,
 'eutt': None,
 'eutf': None,
 'eutm': None,
 'introduce': None,
 'retire': None,
 'lifetime': None,
 'pfail': None,
 'discharge': {Power: True},
 'consume': {Power_in_LiI: True},
 'store': {Power_in_LiI: None},
 'store_loss': {Power_in_LiI: None},
 'sell_cost': None,
 'purchase_cost': None,
 'store_cost': {Power_in_LiI: None},
 'credit': None,
 'penalty': None,
 'basis': None,
 'block': None,
 'citation': None,
 'label': None,
 'trl': None,
 'varying': None,
 'prod_max': None,
 'prod_min': None,
 'named': True,
 'name': 'LiI_d',
 'horizon': h,
 'parameters': [Capacity('LiI_d', 't0')],
 'variables': [capacity('', 'LiI_d', 't0')],
 'constraints': [capacity('', 'LiI_d', 't0')<=M],
 'declared_at': LiI_d,
 'transport': None,
 'transport_l

In [15]:
s.LiI_d.cons()

capacity('', 'LiI_d', 't0')<=M


In [20]:
s.LiI.cons()

capacity('', 'LiI', 't0')<=M
capacity('', 'LiI', 't0')>=1000
capex('LiI', 't0')=3516428*capacity('', 'LiI', 't0')
fopex('LiI', 't0')=87910*capacity('', 'LiI', 't0')
vopex('LiI', 't0')=0*produce('LiI', 't0')


In [23]:
s.Power_in_LiI.__dict__ 

{'discharge': Discharge(Power_in_LiI),
 'consume': Consume(Power_in_LiI),
 'sell_cost': None,
 'purchase_cost': None,
 'credit': None,
 'penalty': None,
 'gwp': None,
 'odp': None,
 'acid': None,
 'eutt': None,
 'eutf': None,
 'eutm': None,
 'basis': None,
 'block': None,
 'label': 'Power generated stored in Lithium-ion battery',
 'citation': None,
 'sell': None,
 'varying': None,
 'price': None,
 'revenue': None,
 'cons_max': None,
 'store_max': None,
 'store_min': None,
 'named': True,
 'name': 'Power_in_LiI',
 'horizon': h,
 'parameters': [Discharge('Power_in_LiI', 'LiI', 't0'),
  Produce('Power_in_LiI', 'LiI', 't0'),
  Consume('Power_in_LiI', 'LiI_d', 't0')],
 'variables': [discharge('Power_in_LiI', 'LiI', 't0'),
  produce('Power_in_LiI', 'LiI', 't0'),
  consume('Power_in_LiI', 'LiI_d', 't0')],
 'constraints': [discharge('Power_in_LiI', 'LiI', 't0')<=M,
  produce('Power_in_LiI', 'LiI', 't0')<=capacity('', 'LiI', 't0'),
  consume('Power_in_LiI', 'LiI_d', 't0')<=M],
 'declared_at': L

In [22]:
s.processes

[LiI_d, LiI, WF, LiI_d, LiI, WF, LiI_d, LiI, WF, LiI_d, LiI, WF]

In [13]:
s.resources 

[H2,
 Solar,
 Wind,
 Power,
 Uranium,
 H2,
 H2_L,
 CO2_AQoff,
 H2O,
 CH4,
 CO2,
 CO2_Vent,
 O2,
 CO2_DAC]

In [14]:
s.processes

[LiI, WF]

In [13]:
s.Power.cons()

discharge('Power', 'LiI', 't0')<=M
discharge('Power', 'LiI', 't0')<=capacity('', 'LiI', 't0')
store('Power', 'LiI', 't0')=2000
store_cost('Power', 'LiI', 't0')=0.4*store('Power', 'LiI', 't0')
store_loss('Power', 'LiI', 't0')=0.2*store('Power', 'LiI', 't0')
discharge('Power', 'WF', 't0')<=M
discharge('Power', 'WF', 't0')<=capacity('', 'WF', 't0')
discharge('Power', 'PV', 't0')<=M
discharge('Power', 'PV', 't0')<=capacity('', 'PV', 't0')
discharge('Power', 'NGCC', 't0')<=M
discharge('Power', 'NGCC', 't0')<=capacity('', 'NGCC', 't0')
discharge('Power', 'H2FC', 't0')<=capacity('', 'H2FC', 't0')
discharge('Power', 'H2FC', 't0')=20
discharge('Power', 'PSH', 't0')<=M
discharge('Power', 'PSH', 't0')<=capacity('', 'PSH', 't0')
store('Power', 'PSH', 't0')=10000
discharge('Power', 'ASMR', 't0')<=M
discharge('Power', 'ASMR', 't0')<=capacity('', 'ASMR', 't0')


In [13]:
s.Power.cons()

store('Power', 'LiI', 't0')=2000
store_cost('Power', 'LiI', 't0')=0.4*store('Power', 'LiI', 't0')
store_loss('Power', 'LiI', 't0')=0.2*store('Power', 'LiI', 't0')
store('Power', 'PSH', 't0')=10000


In [27]:
s.LiI.cons()

capacity('', 'LiI', 't0')=1000
capex('LiI', 't0')=3516428*capacity('', 'LiI', 't0')
fopex('LiI', 't0')=87910*capacity('', 'LiI', 't0')
vopex('LiI', 't0')=0*discharge('LiI', 't0')


In [15]:
s.PV.cons()

capacity('', 'PV', 't0')<=M
capex('PV', 't0')=1333262*capacity('', 'PV', 't0')
fopex('PV', 't0')=22623*capacity('', 'PV', 't0')


In [13]:
s.cons()

discharge('H2', 't0')<=4
discharge('H2', 't0')>=0
discharge('H2', 't1')<=34
discharge('H2', 't1')>=22
discharge('H2', 't2')=Th|Discharge('H2', 't2')|, Th in [DSet|Discharge_lb('H2', 't2')|, DSet|Discharge_ub('H2', 't2')|]
sell_cost('H2', 't0')=Th|Sell_cost('H2', 't0')|*discharge('H2', 't0'), Th in [0, 10]
consume('Solar', 't0')<=M
gwp('Solar', 't0')=50*capacity('Solar', 't0')
gwp('Solar', 't0')=50*consume('Solar', 't0')
gwp('Solar', 't0')=50*discharge('Solar', 't0')
gwp('Solar', 't0')=50*transport('Solar', 't0')
eutt('Solar', 't0')=60*capacity('Solar', 't0')
eutt('Solar', 't0')=60*consume('Solar', 't0')
eutt('Solar', 't0')=60*discharge('Solar', 't0')
eutt('Solar', 't0')=60*transport('Solar', 't0')
consume('Wind', 't2')=DSet|Consume('Wind', 't2')|
consume('Uranium', 't1')=DSet|Consume('Uranium', 't1')|
purchase_cost('Uranium', 't0')=0.3416*consume('Uranium', 't0')
discharge('H2_L', 't0')=Th|Discharge('H2_L', 't0')|, Th in [0, 23]
store('H2_L', 't0')<=M
store('CO2_AQoff', 't0')<=M
consum

In [17]:
len(set(s.constraints))

89

In [13]:
a = {s.Power: {1:{s.Solar: -1.2}, 2: {s.Solar: -1.5}}}

In [21]:
b = {1:{s.Solar: -1.2}, 2: {s.Solar: -1.5}}

In [23]:
[k for k,l in j.items() for i,j in b.items()]

NameError: name 'j' is not defined

In [ ]:
WF.capacity.params[0].ub
WF.capacity.params[0].lb

0

In [ ]:
s.WF.cons()

store_cost('Power', 'LiI', 't0')=0.4*store('Power', 'LiI', 't0')
store('Power', 'LiI', 't0')=2000
store_loss('Power', 'LiI', 't0')=0.2*store('Power', 'LiI', 't0')
store_cost('Power', 'LiI', 't0')=0.4*store('Power', 'LiI', 't0')
store('Power', 'LiI', 't0')=2000
store_loss('Power', 'LiI', 't0')=0.2*store('Power', 'LiI', 't0')
store_cost('Power', 'LiI', 't0')=0.4*store('Power', 'LiI', 't0')
store('Power', 'LiI', 't0')=2000
store_loss('Power', 'LiI', 't0')=0.2*store('Power', 'LiI', 't0')
store_cost('Power', 'LiI', 't0')=0.4*store('Power', 'LiI', 't0')
store('Power', 'LiI', 't0')=2000
store_loss('Power', 'LiI', 't0')=0.2*store('Power', 'LiI', 't0')
store_cost('Power', 'LiI', 't0')=0.4*store('Power', 'LiI', 't0')
store('Power', 'LiI', 't0')=2000
store_loss('Power', 'LiI', 't0')=0.2*store('Power', 'LiI', 't0')
capacity('', 'WF', 't0')=1000
land_use('WF', 't0')=30*capacity('', 'WF', 't0')
odp('WF', 't0')=2900*capacity('', 'WF', 't0')
odp('WF', 't0')=2900*consume('WF', 't0')
odp('WF', 't0')=290

In [13]:
s.WF.params()

Store_cost('Power', 'LiI', 't0')
Store('Power', 'LiI', 't0')
Store_loss('Power', 'LiI', 't0')
Store_cost('Power', 'LiI', 't0')
Store('Power', 'LiI', 't0')
Store_loss('Power', 'LiI', 't0')
Store_cost('Power', 'LiI', 't0')
Store('Power', 'LiI', 't0')
Store_loss('Power', 'LiI', 't0')
Store_cost('Power', 'LiI', 't0')
Store('Power', 'LiI', 't0')
Store_loss('Power', 'LiI', 't0')
Store_cost('Power', 'LiI', 't0')
Store('Power', 'LiI', 't0')
Store_loss('Power', 'LiI', 't0')
Capacity('WF', 't0')
Land_use('WF', 't0')
Odp('WF', 't0')
Gwp('WF', 't0')
Vopex('WF', 't0')
Fopex('WF', 't0')
Capex('WF', 't0')


In [ ]:
Power.consume_scale

In [ ]:
i = tuple(dict.fromkeys(['a', 'b', 'c', 'b']).keys())

In [ ]:
f'{i}' 

"('a', 'b', 'c')"

In [ ]:
WF.eqns()

capex(WF,t0)=1462000*capacity(WF,t0)
fopex(WF,t0)=43000*capacity(WF,t0)
vopex(WF,t0)=4953*discharge(WF,t0)
gwp(WF,t0)=50
odp(WF,t0)=2900
land_use(WF,t0)=30*capacity(WF,t0)
0<=capacity(WF,t0)<=1000
0<=produce(Power,WF,t2)<=capacity(WF,t0)*Produce(Power,WF,t2)


In [ ]:
WF.capacity.indices

[('WF', 't0')]

In [ ]:
PV.eqns()

capex(PV,t0)=1333262*capacity(PV,t0)
fopex(PV,t0)=22623*capacity(PV,t0)
0<=capacity(PV,t0)<=BigM
0<=produce(Power,PV,t0)<=capacity(PV,t0)*1
sell_exp(Power,PV,t0)=30*discharge(Power,PV,t0)
credit(Power,PV,t0)=Credit(Power,PV,t0)*discharge(Power,PV,t0)


In [ ]:
H2.eqns()

0<=produce(H2,t2)<=2000
purchase_exp(H2,H2FC,t0)=2*consume(H2,H2FC,t0)


In [ ]:
NGCC.purchase_cost.eqns()

purchase_exp(CH4,NGCC,t0)=4*consume(CH4,NGCC,t0)
purchase_exp(H2O,NGCC,t0)=20*consume(H2O,NGCC,t0)


In [ ]:
NGCC.eqns()

capex(NGCC,t0)=2158928*capacity(NGCC,t0)
fopex(NGCC,t0)=53320*capacity(NGCC,t0)
vopex(NGCC,t0)=4090*discharge(NGCC,t0)
1<=capacity(NGCC,t0)<=100
0<=produce(Power,NGCC,t0)<=capacity(NGCC,t0)*1
purchase_exp(CH4,NGCC,t0)=4*consume(CH4,NGCC,t0)


In [ ]:
PV.eqns()

capex(PV,t0)=1333262*capacity(PV,t0)
fopex(PV,t0)=22623*capacity(PV,t0)
0<=capacity(PV,t0)<=BigM
0<=produce(Power,PV,t0)<=capacity(PV,t0)*1
sell_exp(Power,PV,t0)=30*discharge(Power,PV,t0)
credit(Power,PV,t0)=Credit(Power,PV,t0)*discharge(Power,PV,t0)


In [ ]:
Power.eqns()

0<=store(Power,LiI,t0)<=capacity(LiI,t0)*2000
0<=produce(Power,LiI,t0)<=capacity(LiI,t0)*1
store_exp(Power,LiI,t0)=0.4*store(Power,LiI,t0)
store_loss(Power,LiI,t2)=0.2
sell_exp(Power,PV,t0)=30*discharge(Power,PV,t0)
credit(Power,PV,t0)=Credit(Power,PV,t0)*discharge(Power,PV,t0)


In [ ]:
Power.sell_cost.indices

[('Power', 'PV', 't0')]

In [ ]:
H2FC.eqns()

capex(H2FC,t0)=1600000.0*capacity(H2FC,t0)
vopex(H2FC,t0)=3.5*discharge(H2FC,t0)
0<=capacity(H2FC,t0)<=1000
0<=produce(Power,H2FC,t0)<=capacity(H2FC,t0)*1
purchase_exp(H2,H2FC,t0)=2*consume(H2,H2FC,t0)


In [ ]:
H2FC.purchase_cost.eqns()

purchase_exp(H2,H2FC,t0)=2*consume(H2,H2FC,t0)


In [ ]:
WF.eqns()

capex(WF,t0)=1462000*capacity(WF,t0)
fopex(WF,t0)=43000*capacity(WF,t0)
vopex(WF,t0)=4953*discharge(WF,t0)
gwp(WF,t0)=50
odp(WF,t0)=2900
land_use(WF,t0)=30*capacity(WF,t0)
0<=capacity(WF,t0)<=1000
0<=produce(Power,WF,t2)<=capacity(WF,t0)*Produce(Power,WF,t2)


In [ ]:
WF.produce.params[0]

Produce(Power,WF,t2)

In [ ]:
WF.store

In [ ]:
Power.eqns()

0<=store(Power,LiI,t0)<=capacity(LiI,t0)*2000
0<=produce(Power,LiI,t0)<=capacity(LiI,t0)*1
store_exp(Power,LiI,t0)=0.4*store(Power,LiI,t0)
store_loss(Power,LiI,t2)=0.2
sell_exp(Power,PV,t0)=30*discharge(Power,PV,t0)
credit(Power,PV,t0)=Credit(Power,PV,t0)*discharge(Power,PV,t0)


In [ ]:
WF.eqns()

capex(WF,t0)=1462000*capacity(WF,t0)
fopex(WF,t0)=43000*capacity(WF,t0)
vopex(WF,t0)=4953*discharge(WF,t0)
gwp(WF,t0)=50
odp(WF,t0)=2900
land_use(WF,t0)=30*capacity(WF,t0)
0<=capacity(WF,t0)<=1000
0<=produce(Power,WF,t2)<=capacity(WF,t0)*Produce(Power,WF,t2)


In [ ]:
PV.eqns()

capex(PV,t0)=1333262*capacity(PV,t0)
fopex(PV,t0)=22623*capacity(PV,t0)
0<=capacity(PV,t0)<=BigM
0<=produce(Power,PV,t0)<=capacity(PV,t0)*1
sell_exp(Power,PV,t0)=30*discharge(Power,PV,t0)
credit(Power,PV,t0)=Credit(Power,PV,t0)*discharge(Power,PV,t0)


In [ ]:
SMRH.eqns()

capex(SMRH,t0)=2520000*capacity(SMRH,t0)
fopex(SMRH,t0)=945000*capacity(SMRH,t0)
vopex(SMRH,t0)=51.5*discharge(SMRH,t0)
land_use(SMRH,t0)=50*capacity(SMRH,t0)
0<=capacity(SMRH,t0)<=1000
0<=produce(H2,SMRH,t0)<=capacity(SMRH,t0)*1


In [ ]:
Solar.eqns()

0<=consume(Solar,t0)<=BigM
gwp(Solar,t0)=50
eutt(Solar,t0)=60


In [49]:
Power.eqns()

0<=store(Power,LiI,t0)<=capacity(LiI,t0)*2000
0<=produce(Power,LiI,t0)<=capacity(LiI,t0)*1
store_exp(Power,LiI,t0)=0.4*store(Power,LiI,t0)
store_loss(Power,LiI,t2)=0.2
sell_exp(Power,PV,t0)=30*discharge(Power,PV,t0)
credit(Power,PV,t0)=Credit(Power,PV,t0)*discharge(Power,PV,t0)


In [50]:
AWE.eqns()

capex(AWE,t0)=1100000.0*capacity(AWE,t0)
fopex(AWE,t0)=16918*capacity(AWE,t0)
incidental(AWE,t0)=20
land_use(AWE,t0)=Th*capacity(AWE,t0), Th in (0,20)
0<=capacity(AWE,t0)<=BigM
0<=produce(H2,AWE,t0)<=capacity(AWE,t0)*1


In [51]:
Power.eqns()

0<=store(Power,LiI,t0)<=capacity(LiI,t0)*2000
0<=produce(Power,LiI,t0)<=capacity(LiI,t0)*1
store_exp(Power,LiI,t0)=0.4*store(Power,LiI,t0)
store_loss(Power,LiI,t2)=0.2
sell_exp(Power,PV,t0)=30*discharge(Power,PV,t0)
credit(Power,PV,t0)=Credit(Power,PV,t0)*discharge(Power,PV,t0)


In [52]:
H2.eqns()

0<=produce(H2,t2)<=2000
purchase_exp(H2,H2FC,t0)=2*consume(H2,H2FC,t0)


In [53]:
Uranium.consume.eqns()

0<=consume(Ur,t1)<=Consume(Ur,Ur,t1)
0<=consume(Ur,ASMR,t0)<=capacity(ASMR,t0)*40


In [54]:
Uranium.eqns()

0<=consume(Ur,t1)<=Consume(Ur,Ur,t1)
purchase_exp(Ur,t0)=0.3416*consume(Ur,t0)


In [55]:
NGCC.eqns()

capex(NGCC,t0)=2158928*capacity(NGCC,t0)
fopex(NGCC,t0)=53320*capacity(NGCC,t0)
vopex(NGCC,t0)=4090*discharge(NGCC,t0)
1<=capacity(NGCC,t0)<=100
0<=produce(Power,NGCC,t0)<=capacity(NGCC,t0)*1
purchase_exp(CH4,NGCC,t0)=4*consume(CH4,NGCC,t0)


In [ ]:
process_list = {LiI, WF, PV, SMRH, SMR, DAC, AWE, H2_L_c, H2_L_d, AQoff_SMR, ASMR, PSH, NGCC}

## Declare locations

In [ ]:
cc = Factor(data = pandas.DataFrame({'a': [2]}), scales= scales, apply_max_scaler= False)

houston = Location(name='HO', processes={LiI, PV, WF, SMRH, AWE, AQoff_SMR}, credit = {PV: 2}, cap_max_localize = {WF: 0.4, AWE: 23}, 
                   credit_factor ={PV: pandas.DataFrame({'a': [2]})}, consume_localize = {Solar: 2}, demand= {Power: (0, 1)}, 
                   land_cost = 10, land_cost_factor= cc, capacity_factor={PV: weather_ho[['dni']], WF: weather_ho[['wind_speed']]}, demand_factor={
                   Power: demand_ho}, discharge_factor={Power: demand_ho}, purchase_cost_factor={CH4: ng_price}, land_max= True, scales=scales, label='Houston')

sandiego = Location(name='SD', processes={LiI, PV, WF, SMRH, AWE, AQoff_SMR}, purchase_cost_localize = {CH4: 222}, 
                    consume_localize = {Solar: 3}, land_cost = 30, land_cost_factor = pandas.DataFrame({'a': [2]}) , 
                    credit = {SMRH: 30, AQoff_SMR: Theta((0, 23))}, credit_factor= {SMRH: pandas.DataFrame({'a': [1 - 0.001*i for i in range(365)]})}, 
                    capacity_factor={PV: weather_sd[['dni']], WF: weather_sd[['wind_speed']]}, demand_factor={
                    Power: demand_sd}, purchase_cost_factor={CH4: ng_price}, discharge_localize= {O2: 0.2}, scales=scales, label='SanDiego')

newyork = Location(name='NY', processes={LiI, H2FC}, land_max = (0, 50000), land_max_factor = pandas.DataFrame({'a': [2]}), cap_max_localize = {PV: 0.2, AWE: 34}, 
                   capacity_factor={PV: weather_ny[['dni']], WF: weather_ny[['wind_speed']]}, demand= {Power: 40}, demand_factor={
                   Power: demand_ny}, credit = {PV: 7}, purchase_cost_localize = {CH4: 2}, scales=scales, label='NewYork')


In [ ]:
PV.cap_max

In [ ]:
PV.factors

In [ ]:
houston.aspect

In [ ]:
houston.land_cost_factor

In [ ]:
# for i in houston.processes | sandiego.processes | newyork.processes:
# # for i in houston.resources | sandiego.resources | newyork.resources:
#     print(i, i.localizations)
#     print(i, i.factors)
    

In [ ]:
Power.ctype

## Declare transports

In [ ]:

# Train_H2 = Transport(name='Train_H2', resources={
#                      H2}, trans_max=bigM, trans_loss=0.03, trans_cost=1.667*10**(-3), label='Railway for hydrogen transportation')
# Grid = Transport(name='Grid', resources={
#                  Power}, trans_max=bigM, trans_loss=0.001, trans_cost=0.5*10**(-3), label='Railroad transport')


Train_H2 = Transport(name='Train_H2', resources={
                     H2}, cap_max=BigM, trans_loss=0.03, capex = 1000, vopex=1.667, fopex = (0,1), acid = 34, eutf =55, label='Railway for hydrogen transportation')
Grid = Transport(name='Grid', resources={
                 H2, Power}, cap_max=BigM, trans_loss=0.001, capex = 500, vopex = 0.444, label='Railroad transport')



In [ ]:
Train_H2.cap_max

## Declare Network

In [ ]:
distance_matrix = [
    [2366], 
    [2620]
]

transport_matrix = [
    [[Train_H2]], 
    [[Grid]],
]
capacity_factor  = {(houston, newyork): {Train_H2: weather_sd[['wind_speed']]}, (sandiego, newyork): {Train_H2: weather_sd[['wind_speed']]}}

network = Network(name='network', scales = scales, land_max= (0,5), land_max_factor= pandas.DataFrame({'a': [5]}),  
                  capacity_factor= capacity_factor, vopex_factor = {(houston, sandiego): {Train_H2: weather_sd[['wind_speed']]}}, sources=[houston, sandiego],
                  sinks=[newyork], distance_matrix=distance_matrix, transport_matrix=transport_matrix)


In [ ]:

scenario = Scenario(name='scenario_full', network=network, scales=scales, label='full_case')


In [ ]:
scenario

In [ ]:
for i in houston.resources:
    print(i.ctype)

In [ ]:
houston.resources_produce

In [ ]:
ResourceType.all()

In [ ]:
ResourceType.resource_level()

In [ ]:
ResourceType.location_level()

In [ ]:
ResourceType.transport_level()

In [ ]:
ProcessType.all()

In [ ]:
ProcessType.process_level()

In [ ]:
ProcessType.location_level()

In [ ]:
LocationType.location_level()

In [ ]:
Power.transport

In [ ]:
Solar.availability

In [ ]:
Power.demand

In [ ]:
[((houston, sandiego), CO2), ((houston, sandiego), H2), ((houston, newyork), CO2)]

In [ ]:
comp_loc_dict = scenario.resources_locations_discharge

In [ ]:
scenario.locations_land

In [ ]:
scenario.locations_locations_land

In [ ]:
comp_loc_ordered_set = [(i, j) for i in comp_loc_dict for j in comp_loc_dict[i]]

In [ ]:
scenario.resources_locations_store

In [ ]:
comp_loc_ordered_set

In [ ]:
comp_loc_ordered_set

In [ ]:
scenario.resources_locations_discharge

In [ ]:
scenario.resources_discharge

In [ ]:
scenario.processes_locations_expenditure

In [ ]:
resources_consume_locations = {resource: set() for resource in scenario.resources_consume}
for resource in scenario.resources_consume:
    for location in scenario.locations:
        if hasattr(location, 'resources_consume'):
            if resource in location.resources_consume:
                resources_consume_locations[resource].add(location)

In [ ]:
resources_consume_locations = {resource: {location for location in scenario.locations if hasattr(location, 'resources_consume') and resource in location.resources_consume} for resource in scenario.resources_consume}

In [ ]:
scenario.resources_consume

In [ ]:
resources_consume_locations = {resource: {location for location in self.locations if hasattr(location, 'resources_consume') and resource in location.resources_consume} for resource in self.resources_consume}

In [ ]:
resources_consume_locations 

In [ ]:
resources_demand_locations = {resource: {location for location in scenario.locations if hasattr(location, 'resources_demand') and resource in location.resources_demand} for resource in scenario.resources_demand}

In [ ]:
scenario.resource_classifications()

In [ ]:
{component: {location for location in getattr(self, 'locations') if hasattr(location, f'{component_set}_{parameter.lower()}') and resource in getattr(location, f'{component_set}_{parameter.lower()}')} for resource in getattr(self, f'{component_set}_{parameter.lower()}')}

In [ ]:
scenario.process_classifications()


In [ ]:
[((houston.name, sandiego.name), CO2.name), ((houston.name, sandiego.name), H2.name), ((houston.name, newyork.name), CO2.name)]

In [ ]:
help(ResourceParamType)

In [ ]:
from pyomo.environ import *
# from energiapy.utils.scale_utils import scale_pyomo_set


In [ ]:
scales1 = TemporalScale(discretization_list=[1, 4], design_scale= 0, scheduling_scale=1)

In [ ]:
type(scales1)

In [ ]:
scales2 = TemporalScale(discretization_list=[2, 2, 2], design_scale= 0, scheduling_scale=2)


In [ ]:
scales1.index_list

In [ ]:
scales2.index_list

In [ ]:
m = ConcreteModel()
m.scales = Set(scales1.list,
                    initialize=scales1.scale)
m.scales_scheduling = scale_pyomo_set(
        instance=m, scale_level=scales1.scheduling_scale)
m.scales_design = scale_pyomo_set(
        instance=m, scale_level=scales1.design_scale)



In [ ]:
scales.scheduling_scale

In [ ]:
# m.a = Set(initialize = [(i, j) for i in resources_consume_locations for j in resources_consume_locations[i]])
# m.b = Set(initialize = [(i.name, j.name) for i in scenario.resources_locations_consume for j in scenario.resources_locations_consume[i]])
# m.c = Set(initialize = [((houston.name, sandiego.name), CO2.name), ((houston.name, sandiego.name), H2.name), ((houston.name, newyork.name), CO2.name)])
# m.A = Var(m.a, m.scales_scheduling, within = NonNegativeReals)
# m.B = Var(m.b, m.scales_design, within = NonNegativeReals)
# m.C = Var(m.c, m.scales_scheduling, within = NonNegativeReals)

In [ ]:
from pyomo.environ import * 
m = ConcreteModel()

In [ ]:
m.d = Set(initialize = [(Train_H2.name, (houston.name, sandiego.name)), (Grid.name, (houston.name, sandiego.name)), (Grid.name, (houston.name, newyork.name))])
m.D = Var(m.d, m.scales_design, within = NonNegativeReals)


In [ ]:
m.f = Set(initialize = [Train_H2.name, Grid.name])
m.F = Var(m.f, m.scales_design, within = NonNegativeReals)


In [ ]:
m.g = Set(initialize = [(CO2.name, (houston.name, sandiego.name)), (Power.name, (houston.name, sandiego.name)), (H2.name, (houston.name, newyork.name))])

In [ ]:
m.G = Var(m.g, m.scales_scheduling, within = NonNegativeReals)

In [ ]:
m.G.pprint()

In [ ]:
m = ConcreteModel()


In [ ]:
import gurobipy
m = gurobipy.read('example.lp')

In [ ]:
import pandas 

In [ ]:
a = pandas.DataFrame(data= {'a': [1, 2, 3]})

In [ ]:
b = 

In [ ]:
class LPFileCreator:
    def __init__(self, objective, constraints, bounds, file_name="problem.lp"):
        self.objective = objective
        self.constraints = constraints
        self.bounds = bounds
        self.file_name = file_name
    
    def create_lp_file(self):
        content = self._generate_lp_content()
        with open(self.file_name, "w") as file:
            file.write(content)
    
    def _generate_lp_content(self):
        lp_content = []
        
        # Add Objective Function
        lp_content.append("\\* Problem Statement *\\")
        lp_content.append("\nMaximize")
        lp_content.append(f" obj: {self.objective}\n")
        
        # Add Constraints
        lp_content.append("Subject To")
        for i, constraint in enumerate(self.constraints, start=1):
            lp_content.append(f" c{i}: {constraint}")
        lp_content.append("")
        
        # Add Bounds
        lp_content.append("Bounds")
        for bound in self.bounds:
            lp_content.append(f" {bound}")
        lp_content.append("")
        
        # Add End
        lp_content.append("End")
        
        return "\n".join(lp_content)


In [ ]:
a = [7, 2, 3].sort()


In [ ]:
a

In [ ]:
sorted(a)

In [ ]:
houston.resources_consume

In [ ]:
[f'cons_{i.name}_' for i in houston.resources_consume]

In [ ]:
def tuples_to_strings(tuple_list):
    return ['_'.join(map(str, t)) for t in tuple_list]

In [ ]:
scales = TemporalScale([1,4], design_scale= 0, scheduling_scale=1)

In [ ]:
var = ['cons'] #, 'sell']

In [ ]:
res_list = ['res1', 'res2', 'res3']

In [ ]:
time_index = tuples_to_strings(scales.index_list[1])

In [ ]:
value = {'res1': 5, 'res2': 7, 'res3': 8}

In [ ]:
cons = list()

In [ ]:
for i,j,k in product(var, res_list, time_index):
    cons.append(f'{i}_{j}_{k} <= {value[j]}')

In [ ]:
ooo = {'res1': 7, 'res2': 5, 'res3': 9}

In [ ]:
k = [f'{ooo[i]} {i}' for i in res_list]

In [ ]:
obj = ' + '.join(k)
obj

In [ ]:
for i,j,k in product(var, res_list, time_index):


In [ ]:
bou = [f'0 <= {i}' for i in var]

In [ ]:

# Define the problem components
objective = "x + 2 y"
constraints = [
    "x + y >= 1",
    "x <= 2",
    "y <= 2"
]
bounds = [
    "0 <= x",
    "0 <= y"
]

# Create the LP file
# creator = LPFileCreator(objective, constraints, bounds, file_name="example.lp")

creator = LPFileCreator(obj, cons, bou, file_name="example.lp")
creator.create_lp_file()
print("LP file 'example.lp' created successfully.")

In [ ]:
import gurobipy as gb 

a = gb.read('example.lp')

In [ ]:
a.optimize()

In [ ]:
Maximize
  x + y + z
Subject To
  c0: x + y = 1
  c1: x + 5 y + 2 z <= 10
  qc0: x + y + [ x ^ 2 - 2 x * y + 3 y ^ 2 ] <= 5
Bounds
  0 <= x <= 5
  z >= 2
Generals
  x y z
End


In [ ]:
H2O.purchase_cost

In [ ]:
H2O.cons_max

In [ ]:
m.optimize()

In [ ]:
m.getVars()

In [ ]:
plot_scenario.capacity_factor(scenario = scenario, location= houston, process= PV, fig_size= (9,5), color= 'orange')


![cap fac pv](plots/cf_ho.png)

In [ ]:
plot_scenario.capacity_factor(scenario = scenario, location= sandiego, process= WF, fig_size= (9,5), color= 'blue')


![cap fac wf](plots/wf_sd.png)

In [ ]:
plot_scenario.demand_factor(scenario = scenario, location= newyork, resource= Power, fig_size= (9,5), color= 'red')


![df ny](plots/df_ny.png)

## Optimize for minimum cost

In [ ]:
milp_cost = formulate(scenario= scenario, constraints={Constraints.COST, Constraints.INVENTORY, Constraints.PRODUCTION, Constraints.RESOURCE_BALANCE, Constraints.TRANSPORT, Constraints.MODE, Constraints.NETWORK},  objective=Objective.COST)

**Ensure no discharge of power in Houston and San Diego**

While the demand for power is zero in these locations, Power is still a dischargeable resource. Hence, the discharge bounds need to be set to zero.
If not set to zero, the system chooses to dispense power instead of establishing storage networks.

In [ ]:

milp_cost.constraint_specific_location_discharge_ho = constraint_specific_location_discharge(
        instance=milp_cost, network_scale_level=0, bounds={Power: 0}, location = houston)
milp_cost.constraint_specific_location_discharge_sd = constraint_specific_location_discharge(
        instance=milp_cost, network_scale_level=0, bounds={Power: 0}, location = sandiego)


In [ ]:

results_cost = solve(scenario = scenario, instance= milp_cost, solver= 'gurobi', name="results_cost", print_solversteps = True)


**Schedule for transport of hydrogen**

In [ ]:
plot_results.transport(results=results_cost, source='SD', sink='NY', resource='H2', transport='Train_H2')


![sch t sdny](plots/sch_sdny.png)

In [ ]:
plot_results.transport(results=results_cost, source='HO', sink='NY', resource='H2', transport='Train_H2')

![sch hony](plots/sch_hony.png)

**Cost contribution by technology**

In [ ]:
plot_results.cost(results= results_cost, x = CostX.PROCESS_WISE, y = CostY.CAPEX, location= 'HO', fig_size= (8,6))
plot_results.cost(results= results_cost, x = CostX.PROCESS_WISE, y = CostY.VOPEX, location= 'HO', fig_size= (8,6))
plot_results.cost(results= results_cost, x = CostX.PROCESS_WISE, y = CostY.FOPEX, location= 'HO', fig_size= (8,6))
plot_results.cost(results= results_cost, x = CostX.PROCESS_WISE, y = CostY.TOTAL, location= 'HO', fig_size= (8,6))

![total cost ho](plots/total_ho.png)

**Location-wise cost contribution by technology**

In [ ]:

plot_results.cost(results= results_cost, x = CostX.LOCATION_WISE, y = CostY.CAPEX, fig_size= (8,6))
plot_results.cost(results= results_cost, x = CostX.LOCATION_WISE, y = CostY.VOPEX, fig_size= (8,6))
plot_results.cost(results= results_cost, x = CostX.LOCATION_WISE, y = CostY.FOPEX, fig_size= (8,6))
plot_results.cost(results= results_cost, x = CostX.LOCATION_WISE, y = CostY.TOTAL, fig_size= (8,6))

![total lw](plots/total_lw.png)

**Inventory and production schedules**

In [ ]:
plot_results.schedule(results= results_cost, y_axis= 'Inv', component= 'LiI_Power_stored', location = 'SD', fig_size= (9,5), color = 'steelblue')

![inv lii](plots/lii_inv.png)

In [ ]:
plot_results.schedule(results= results_cost, y_axis= 'P', component= 'PV', location = 'SD', fig_size= (9,5), color = 'steelblue')

![pv p](plots/pv_p.png)

## Optimize to maximize resource discharge 

In [ ]:

milp_demand = formulate(scenario=scenario, constraints={Constraints.COST, Constraints.INVENTORY, Constraints.PRODUCTION,
                        Constraints.RESOURCE_BALANCE, Constraints.TRANSPORT, Constraints.MODE, Constraints.NETWORK},  objective=Objective.MAX_DISCHARGE, objective_resource=Power)


In [ ]:

milp_demand.constraint_specific_location_discharge_ho = constraint_specific_location_discharge(
        instance=milp_demand, network_scale_level=0, bounds={Power: 0}, location = houston)
milp_demand.constraint_specific_location_discharge_sd = constraint_specific_location_discharge(
        instance=milp_demand, network_scale_level=0, bounds={Power: 0}, location = sandiego)


In [ ]:

results_demand = solve(scenario=scenario, instance=milp_demand, solver='gurobi',
                       name="results_demand", print_solversteps=True)


**Schedule for transport of hydrogen**

In [ ]:
plot_results.transport(results=results_demand, source='SD', sink='NY', resource='H2', transport='Train_H2')


![sch t sdny2](plots/sch_sdny2.png)

In [ ]:
plot_results.transport(results=results_demand, source='HO', sink='NY', resource='H2', transport='Train_H2')

![sch t hony](plots/sch_hony2.png)